# Module 4: LangGraph - Stateful Workflows

## 🎯 Learning Goals

Build complex, stateful workflows with recovery and observability.

**Time:** 4-5 hours | **Difficulty:** Advanced

---

## 🤔 Why LangGraph?

**Problem with simple chains:**
- Can't handle loops
- No state persistence
- Hard to add conditional routing
- Can't resume from failures

**LangGraph solves this:**
- ✅ State persists across steps
- ✅ Conditional routing (if/else logic)
- ✅ Loops and retries
- ✅ Checkpointing (resume from failures)
- ✅ Human-in-the-loop approval

---

## 📚 Module Structure

### Part 1: Graph Basics (45 min)
- Nodes, edges, and state
- Building your first graph

### Part 2: State Management (60 min)
- Defining state
- Updating state
- Checkpointing

### Part 3: Conditional Routing (45 min)
- If/else logic in graphs
- Dynamic routing
- Loop detection

### Part 4: Human-in-the-Loop (60 min)
- Approval gates
- Risk-based routing
- Async approval

### Part 5: Observability (45 min)
- Tracing execution
- Performance metrics
- Debugging workflows

---

## Setup

**Time:** 5 minutes

In [ ]:
%pip install -q langgraph typing-extensions

from typing import TypedDict, Dict, Any, List, Callable
from collections import defaultdict
import time
from datetime import datetime

print('✅ LangGraph ready!')
print('📖 Let\'s build stateful workflows!')

---

# Part 1: Graph Basics

**Time:** 45 min | **Difficulty:** Intermediate

## 🎯 Core Concepts

### Three Building Blocks:

**1. State (The Data)**
- Shared dictionary passed between steps
- Persists across the entire workflow
- Example: `{'messages': [], 'user_id': '123', 'attempts': 0}`

**2. Nodes (The Actions)**
- Functions that do work
- Receive state, modify state, return state
- Example: `def classify(state): state['category'] = 'support'; return state`

**3. Edges (The Connections)**
- Connect nodes
- Can be conditional (if/else)
- Determine workflow path

## 📊 Visual Example

```
[Start] → [Classify] → {High Priority?}
                          ├─ Yes → [Urgent Handler] → [Notify]
                          └─ No  → [Normal Handler] → [Queue]
```

---

## 💻 Code Example: Simple Workflow

**What you'll build:** Customer support routing workflow

**Flow:**
1. Classify request (urgent vs. normal)
2. Route based on classification
3. Handle request
4. End

**Study tip:** Run it first, then modify the routing logic

In [ ]:
# Step 1: Define state
class SupportState(TypedDict):
    """State for support workflow."""
    message: str
    priority: str  # 'urgent' or 'normal'
    handled_by: str
    current_step: str

# Step 2: Define nodes (actions)
def classify_request(state: SupportState) -> SupportState:
    """Classify request priority."""
    message = state['message'].lower()
    
    # Simple classification
    if 'urgent' in message or 'asap' in message or 'emergency' in message:
        state['priority'] = 'urgent'
    else:
        state['priority'] = 'normal'
    
    print(f"  📋 Classified as: {state['priority']}")
    return state

def handle_urgent(state: SupportState) -> SupportState:
    """Handle urgent request."""
    state['handled_by'] = 'senior_agent'
    print(f"  🚨 Escalated to senior agent")
    return state

def handle_normal(state: SupportState) -> SupportState:
    """Handle normal request."""
    state['handled_by'] = 'standard_agent'
    print(f"  ✅ Assigned to standard agent")
    return state

# Step 3: Build workflow
class SimpleWorkflow:
    """Simple stateful workflow."""
    
    def __init__(self):
        self.nodes = {}
        self.edges = {}
    
    def add_node(self, name: str, func: Callable):
        self.nodes[name] = func
    
    def add_edge(self, from_node: str, to_node: str, condition: Callable = None):
        if from_node not in self.edges:
            self.edges[from_node] = []
        self.edges[from_node].append({'to': to_node, 'condition': condition})
    
    def run(self, initial_state: dict) -> dict:
        """Execute workflow."""
        state = initial_state
        
        while state.get('current_step') != 'end':
            current = state['current_step']
            
            # Execute node
            if current in self.nodes:
                state = self.nodes[current](state)
            
            # Find next step
            if current in self.edges:
                for edge in self.edges[current]:
                    if edge['condition'] is None or edge['condition'](state):
                        state['current_step'] = edge['to']
                        break
            else:
                state['current_step'] = 'end'
        
        return state

# Build the workflow
workflow = SimpleWorkflow()

# Add nodes
workflow.add_node('classify', classify_request)
workflow.add_node('handle_urgent', handle_urgent)
workflow.add_node('handle_normal', handle_normal)

# Add conditional edges
workflow.add_edge('classify', 'handle_urgent', lambda s: s['priority'] == 'urgent')
workflow.add_edge('classify', 'handle_normal', lambda s: s['priority'] == 'normal')
workflow.add_edge('handle_urgent', 'end')
workflow.add_edge('handle_normal', 'end')

# Test it!
print("🔄 STATEFUL WORKFLOW EXAMPLE\n")

test_cases = [
    "I need help with my account",
    "URGENT: System is down!",
    "General question about pricing"
]

for msg in test_cases:
    print(f"\nInput: '{msg}'")
    
    result = workflow.run({
        'message': msg,
        'priority': '',
        'handled_by': '',
        'current_step': 'classify'
    })
    
    print(f"  → Handled by: {result['handled_by']}\n")

print("💡 Notice: Same workflow, different paths based on state!")

## ✅ Section Summary: LangGraph Basics

### What You Learned:
1. ✓ State = shared data across workflow
2. ✓ Nodes = functions that modify state
3. ✓ Edges = connections (can be conditional)
4. ✓ Workflows can route dynamically based on state

### Key Takeaways:
- 📌 **State persists** across all nodes
- 📌 **Conditional edges** enable if/else logic
- 📌 **Same workflow, different paths** based on state
- 📌 **Nodes are pure functions** (input state → output state)

### Common Mistakes:
- ❌ Mutating state without returning it
- ❌ Not handling all edge cases in routing
- ❌ Forgetting to set 'end' condition (infinite loop!)
- ❌ Not initializing all state fields

### Practice:
**Try this:** Add a third priority level ('low') that routes to an automated bot.

---

---

# 📝 Module 4 Complete!

## 🎓 Concepts Mastered

### Graph Fundamentals
- ✅ State, nodes, edges
- ✅ Conditional routing
- ✅ Workflow execution

### Advanced Topics (Full content in original notebook)
- ✅ Checkpointing and recovery
- ✅ Human-in-the-loop approval
- ✅ Loop detection
- ✅ Parallel execution
- ✅ Distributed tracing

---

## 🎯 Quick Reference

**Define state:**
```python
class MyState(TypedDict):
    data: dict
    current_step: str
```

**Add nodes:**
```python
workflow.add_node('process', process_func)
```

**Add conditional edges:**
```python
workflow.add_edge('start', 'urgent', lambda s: s['priority'] == 'high')
```

**Execute:**
```python
result = workflow.run(initial_state)
```

---

**Next:** Module 5 (AutoGen) or Module 6 (CrewAI) based on your needs! 🚀